In [1]:
import pandas as pd
import plotly.graph_objects as go
from io import StringIO

In [2]:
results = """num_threads,size,seq,par_red,par_crit
1,100,1.84774e-06,2.98023e-06,6.25849e-06
1,1000,2.96235e-05,3.03984e-05,6.9499e-05
1,10000,0.000870883,0.000821382,0.00121763
1,100000,0.0490145,0.0471202,0.0499461
2,100,1.37091e-06,2.12491e-05,8.85129e-06
2,1000,2.22623e-05,1.08778e-05,0.000114024
2,10000,0.000579745,0.000391245,0.00142214
2,100000,0.047201,0.035092,0.0388059
4,100,1.2219e-06,2.42591e-05,1.67489e-05
4,1000,2.17557e-05,5.87106e-06,0.000172883
4,10000,0.000579864,0.000226885,0.00172412
4,100000,0.0481128,0.0204418,0.0248393
8,100,1.75834e-06,3.68655e-05,1.78814e-05
8,1000,2.87592e-05,5.99027e-06,0.000183523
8,10000,0.000697762,0.000145882,0.00253126
8,100000,0.0481544,0.0146812,0.0508929
16,100,1.99676e-06,0.00153977,0.000179738
16,1000,3.3766e-05,3.87728e-05,0.00201586
16,10000,0.000899881,0.000267625,0.0226844
16,100000,0.0645413,0.0234869,0.216839
32,100,2.5034e-06,0.00251588,0.000275493
32,1000,3.76105e-05,0.000223368,0.00322688
32,10000,0.000895649,0.00295976,0.0333724
32,100000,0.0666359,0.0197409,0.609156
64,100,3.75509e-06,0.00583339,0.000689864
64,1000,5.57601e-05,0.000878513,0.0109569
64,10000,0.00815776,0.00643548,0.170112
64,100000,0.0765735,0.0323107,1.29195"""

In [3]:
results_file = StringIO(results)
df = pd.read_csv(results_file)
df.head()

,num_threads,size,seq,par_red,par_crit
0,1,100,0.000002,0.000003,0.000006
1,1,1000,0.000030,0.000030,0.000069
2,1,10000,0.000871,0.000821,0.001218
3,1,100000,0.049015,0.047120,0.049946
4,2,100,0.000001,0.000021,0.000009


In [4]:
algorithms = ("seq", "par_red", "par_crit")

speedup = df.copy()
for algo in algorithms:
    speedup[algo] = df["seq"] / df[algo]

speedup.head()

,num_threads,size,seq,par_red,par_crit
0,1,100,1.0,0.619999,0.295237
1,1,1000,1.0,0.974509,0.426244
2,1,10000,1.0,1.060266,0.715228
3,1,100000,1.0,1.040201,0.981348
4,2,100,1.0,0.064516,0.154883


In [14]:
surfaces = []
for algo in algorithms:
    algo_df = speedup[["num_threads", "size", algo]]
    algo_df = algo_df.pivot(index="num_threads", columns="size", values=algo)
    opacity = 0.5 if algo == "seq" else 1
    surf = go.Surface(y=algo_df.index, x=algo_df.columns, z=algo_df, name=algo, opacity=opacity)
    surfaces.append(surf)

fig = go.Figure(data=surfaces)
fig.update_layout(title="Speedup comparison", legend_title_text='Algorithm', scene=dict(xaxis_title="Size", xaxis_type="log", yaxis_title="# of threads", yaxis_type="log", zaxis_title="Speedup"))
fig.write_html("pi.html")

In [11]:
algo_df = speedup[["num_threads", "size", "par_red"]]
algo_df = algo_df.pivot(index="num_threads", columns="size", values="par_red")
algo_df

size,100,1000,10000,100000
num_threads,,,,
1,0.619999,0.974509,1.060266,1.040201
2,0.064516,2.046581,1.481795,1.345064
4,0.050369,3.705583,2.555762,2.353648
8,0.047696,4.800986,4.783058,3.280004
16,0.001297,0.870868,3.362470,2.747970
32,0.000995,0.168379,0.302609,3.375525
64,0.000644,0.063471,1.267623,2.369912
